### Problem 3. 주어진  사진  샘플  8장에  원통형  투영을  적용해  하나의 이미지로 합치기

In [24]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
img1 = cv2.imread('./image/problem_3/1.jpg')
img2 = cv2.imread('./image/problem_3/2.jpg')
img3 = cv2.imread('./image/problem_3/3.jpg')
img4 = cv2.imread('./image/problem_3/4.jpg')
img5 = cv2.imread('./image/problem_3/5.jpg')
img6 = cv2.imread('./image/problem_3/6.jpg')
img7 = cv2.imread('./image/problem_3/7.jpg')
img8 = cv2.imread('./image/problem_3/8.jpg')


img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
img5 = cv2.cvtColor(img5, cv2.COLOR_BGR2RGB)
img6 = cv2.cvtColor(img6, cv2.COLOR_BGR2RGB)
img7 = cv2.cvtColor(img7, cv2.COLOR_BGR2RGB)
img8 = cv2.cvtColor(img8, cv2.COLOR_BGR2RGB)

In [25]:
from PIL import Image
from PIL.ExifTags import TAGS
import numpy as np

def get_camera_matrices(image_path, sensor_width=23.5, sensor_height=15.6):
    img = Image.open(image_path)
    exif_data = img._getexif()

    if exif_data is None:
        print("No EXIF data found.")
        return None, None
    
    focal_length = None
    focal_length_35mm = None
    image_width, image_height = img.size

    for tag, value in exif_data.items():
        tag_name = TAGS.get(tag, tag)
        
        # FocalLength (기본)
        if tag_name == 'FocalLength':
            if isinstance(value, tuple):
                focal_length = value[0] / value[1]
            else:
                focal_length = float(value)

        # FocalLengthIn35mmFilm (35mm 환산)
        if tag_name == 'FocalLengthIn35mmFilm':
            focal_length_35mm = int(value)

    # 기본 Focal Length가 있으면 사용하고, 없으면 35mm 환산값 사용
    if focal_length_35mm is not None:
        # 35mm 환산을 기준으로 초점 거리 계산
        focal_length = (focal_length_35mm / 35.0) * sensor_width
    
    # 기본 Focal Length도 없는 경우
    if focal_length is None:
        print("No valid focal length found in EXIF data.")
        return None, None

    # K 계산 (float32 타입으로 설정)
    fx = (focal_length / sensor_width) * image_width
    fy = (focal_length / sensor_height) * image_height  # 수직 초점 거리 계산
    cx = image_width / 2
    cy = image_height / 2

    K = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 1]
    ], dtype=np.float32)

    # 기본적으로 R은 단위 행렬로 시작
    R = np.eye(3, dtype=np.float32)

    return K, R


In [26]:
# 각 이미지의 주요 메타데이터(촬영 시간, 모델명 등) 비교
def get_metadata(image_path):
    img = Image.open(image_path)
    exif_data = img._getexif()
    
    if exif_data is None:
        print("No EXIF data found.")
        return None
    
    metadata = {}
    for tag, value in exif_data.items():
        tag_name = TAGS.get(tag, tag)
        metadata[tag_name] = value

    return metadata
def print_metadata_summary(meta, idx):
    print(f"Image {idx}:")
    for key, value in meta.items():
        if key not in ['PrintImageMatching','UserComment','MakerNote','GPSInfo']:

           print(f"  {key}: {value}")
print_metadata_summary(get_metadata('./image/problem_2/3.jpg'), 1)
print_metadata_summary(get_metadata('./image/problem_2/4.jpg'), 2)
print_metadata_summary(get_metadata('./image/problem_2/5.jpg'), 3)

Image 1:
  ResolutionUnit: 2
  ExifOffset: 358
  ImageDescription:                                
  Make: SONY
  Model: NEX-5N
  Software: GIMP 2.8.3
  Orientation: 1
  DateTime: 2014:04:09 18:33:06
  YCbCrPositioning: 2
  XResolution: 350.0
  YResolution: 350.0
  ExifVersion: b'0230'
  ComponentsConfiguration: b'\x01\x02\x03\x00'
  CompressedBitsPerPixel: 2.0
  DateTimeOriginal: 2013:01:22 11:50:44
  DateTimeDigitized: 2013:01:22 11:50:44
  BrightnessValue: 8.5171875
  ExposureBiasValue: 0.0
  MaxApertureValue: 3.6171875
  MeteringMode: 5
  LightSource: 0
  Flash: 16
  FocalLength: 18.0
  ColorSpace: 1
  ExifImageWidth: 2000
  SceneCaptureType: 0
  ExifImageHeight: 1329
  Contrast: 0
  Saturation: 0
  Sharpness: 0
  FileSource: b'\x03'
  ExposureTime: 0.005
  ExifInteroperabilityOffset: 39892
  FNumber: 7.1
  SceneType: b'\x01'
  ExposureProgram: 2
  CustomRendered: 0
  ISOSpeedRatings: 100
  ExposureMode: 0
  FlashPixVersion: b'0100'
  WhiteBalance: 0
  FocalLengthIn35mmFilm: 27
Ima

In [27]:
import cv2
import numpy as np
import itertools
# 2. 키포인트 검출 및 매칭
def detect_keypoints(image):
    sift = cv2.SIFT_create()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if image.ndim == 3 else image
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors

# 3. 매칭된 키포인트 쌍 및 score 계산
from typing import Tuple
def compute_matching(kp1, des1, kp2, des2, ratio=0.7, max_distance=80.0, use_ransac=True):
    # 특징점이 없을 때
    if des1 is None or des2 is None:
        return 0, [], np.eye(3)
    
    # 특징점 매칭
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Lowe's ratio test
    good_matches = [m[0] for m in matches if len(m) == 2 and m[0].distance < ratio * m[1].distance]

    # 거리 기반 필터링
    good_matches = [m for m in good_matches if m.distance < max_distance]
    
    # 호모그래피 검증
    if use_ransac and len(good_matches) >= 4:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
        good_matches = [m for i, m in enumerate(good_matches) if mask[i]]
        score = len(good_matches)
        return score, good_matches, H
    else:
        raise ValueError("Not enough matches found to compute homography.")


In [28]:
class MatchData:
    def __init__(self):
        '''
        MatchData 객체 초기화
        데이터는 각각 순서대로 pair, kp1, kp2, score, good_matches, H로 구성됨
        pair: (img1, img2) 이미지 index 쌍
        kp1, kp2: 각각의 이미지에서 검출된 키포인트
        score: 매칭된 키포인트 쌍의 수
        good_matches: 매칭된 키포인트 쌍
        H: 호모그래피 행렬
        '''
        self.data = []

    def add(self, pair,kp1,kp2, score, good_matches, H):
        if not isinstance(pair, tuple) or len(pair) != 2:
            raise ValueError("Pair must be a tuple of (img1, img2).")
        if type(pair[0]) != int or type(pair[1]) != int:
            raise ValueError("Pair elements must be integers.")
        if not isinstance(kp1, tuple) or not isinstance(kp2, tuple):
            raise ValueError("Keypoints must be lists.")
        if not isinstance(score, int):
            raise ValueError("Score must be an integer.")
        if not isinstance(good_matches, list):
            raise ValueError("Good matches must be a list.")
        if not isinstance(H, np.ndarray) or H.shape != (3, 3):
            raise ValueError("H must be a 3x3 numpy array.")
        if score < 0:
            raise ValueError("Score must be non-negative.")
        if len(good_matches) < 4:
            raise ValueError("At least 4 good matches are required to compute homography.")
        self.data.append((pair,kp1,kp2, score, good_matches, H))

    def __setitem__(self, idx, value):
        if not isinstance(value, tuple) or len(value) !=6:
            raise ValueError("Value must be a tuple of (pair, kp1, kp2, score, good_matches, H).")

    def sort(self, reverse=False):
        self.data.sort(key=lambda x: x[3], reverse=reverse) # sort by score
    
    def index(self, score):
        """
        Return the index of the first matching item with the given score.
        """
        for idx, item in enumerate(self.data):
            if item[3] == score:
                return idx
            if type(score) not in (int, float):
                raise TypeError("Score must be an integer or float.")
        raise ValueError(f"Score {score} not found in MatchData.") 
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self.data):
            raise IndexError("Index out of range.")
        return self.data[idx]   

In [29]:
class Blender:
    def __init__(self, blending_method="none"):
        self.blending_method = blending_method.lower()
        self.H_matrices = None
        
        # 허용된 블렌딩 방법 체크
        if self.blending_method not in ["none", "simple"]:
            raise ValueError("잘못된 블렌딩 방법입니다. 'none' 또는 'simple' 중 하나를 선택하세요.")

    def set_h_matrices(self, H_matrices):
        """
        H_matrices를 설정하는 메서드
        """
        if not isinstance(H_matrices, dict):
            raise ValueError("H_matrices는 딕셔너리여야 합니다.")
        self.H_matrices = H_matrices

    def blend_images(self, images):
        """
        선택된 블렌딩 방법에 따라 이미지를 블렌딩
        
        Parameters:
        - images (list): 블렌딩할 이미지 리스트
        
        Returns:
        - np.ndarray: 블렌딩된 최종 이미지
        """
        if self.H_matrices is None:
            raise ValueError("H_matrices가 설정되지 않았습니다. 먼저 set_h_matrices를 호출하세요.")
        
        if self.blending_method == "none":
            return self.none_blend(images)
        elif self.blending_method == "simple":
            return self.simple_blend(images)
        else:
            raise ValueError("지원되지 않는 블렌딩 방법입니다.")
        
    def none_blend(self, images):
        """
        블렌딩 없이 단순히 첫 번째 이미지를 정렬하여 반환
        """
        # 첫 번째 앵커 인덱스 설정 (단위 행렬이 할당된 인덱스)
        anchor_index = list(self.H_matrices.keys())[0]
        anchor_img = images[anchor_index]
        H_anchor = self.H_matrices[anchor_index]

        # 출력 캔버스 크기 계산
        corners = []
        for idx, H in self.H_matrices.items():
            h, w = images[idx].shape[:2]
            corners += [H @ np.array([0, 0, 1]),
                        H @ np.array([w, 0, 1]),
                        H @ np.array([0, h, 1]),
                        H @ np.array([w, h, 1])]
        
        # 코너 좌표 정규화
        corners = np.array(corners)
        corners /= corners[:, 2].reshape(-1, 1)
        min_x, min_y = np.min(corners[:, :2], axis=0).astype(int)
        max_x, max_y = np.max(corners[:, :2], axis=0).astype(int)
        width = max_x - min_x
        height = max_y - min_y

        # 오프셋 변환
        translation = np.array([[1, 0, -min_x], [0, 1, -min_y], [0, 0, 1]], dtype=np.float32)

        # 앵커 이미지 왜곡 (단위 행렬 적용)
        stitched_image = cv2.warpPerspective(anchor_img, translation @ H_anchor, (width, height))

        # 다른 이미지 추가 (단순 블렌딩)
        for idx, H in self.H_matrices.items():
            if idx == anchor_index:
                continue
            warped_img = cv2.warpPerspective(images[idx], translation @ H, (width, height))
            mask = (warped_img.sum(axis=2) > 0).astype(np.uint8)
            stitched_image[mask > 0] = warped_img[mask > 0]

        return stitched_image
    
    def simple_blend(self, images):
        """
        간단한 평균 블렌딩
        """
        # 출력 캔버스 크기 계산
        corners = []
        for idx, H in self.H_matrices.items():
            h, w = images[idx].shape[:2]
            corners += [H @ np.array([0, 0, 1]),
                        H @ np.array([w, 0, 1]),
                        H @ np.array([0, h, 1]),
                        H @ np.array([w, h, 1])]
        
        corners = np.array(corners)
        corners /= corners[:, 2].reshape(-1, 1)
        min_x, min_y = np.min(corners[:, :2], axis=0).astype(int)
        max_x, max_y = np.max(corners[:, :2], axis=0).astype(int)
        width = max_x - min_x
        height = max_y - min_y
        
        # 오프셋 변환 행렬
        translation = np.array([[1, 0, -min_x], [0, 1, -min_y], [0, 0, 1]], dtype=np.float32)

        # 블렌딩할 최종 이미지 초기화
        blended_image = np.zeros((height, width, 3), dtype=np.float32)
        blend_count = np.zeros((height, width), dtype=np.float32)

        # 모든 이미지의 픽셀 값 합산
        for idx, H in self.H_matrices.items():
            warped_img = cv2.warpPerspective(images[idx], translation @ H, (width, height))
            mask = (warped_img.sum(axis=2) > 0).astype(np.float32)
            blended_image += warped_img * mask[..., np.newaxis]
            blend_count += mask
        
        # 정규화하여 최종 블렌딩 이미지 생성
        blended_image /= np.maximum(blend_count[..., np.newaxis], 1)

        return blended_image.astype(np.uint8)

In [30]:
class ImageStitcher(Blender):
    def __init__(self, blending_method="none"):
        Blender.__init__(self, blending_method)
        self.H_matrices = None

    def __call__(self, match_data, images):
        # 매칭 데이터가 비어 있는지 확인
        if len(match_data) == 0:
            raise ValueError("매칭 데이터가 비어 있습니다.")
        
        # 매칭 데이터 정렬 (내림차순)
        match_data.sort(reverse=True)
        
        # 앵커 이미지와 그 인덱스 찾기
        best_index = self.find_anchor(match_data)
        
        # 호모그래피 그래프 생성
        self.H_matrices = self.build_homography(best_index, match_data, images)
        
        # 블렌딩 수행
        return self.blend_images(images)
    
    def find_anchor(self, match_data):
        # 가장 많은 매칭을 가진 이미지와 해당 인덱스 찾기
        best_score = max(match_data.data, key=lambda x: x[3])[3]
        best_index = match_data.index(best_score)
        return best_index
    
    def build_homography(self, best_index, match_data, images):
        """
        호모그래피 그래프 생성
        
        Parameters:
        - best_index (int): 앵커 이미지의 인덱스
        - match_data (MatchData): 이미지 쌍과 호모그래피를 포함한 매칭 데이터
        - images (list): 블렌딩할 이미지 리스트
        
        Returns:
        - dict: 각 이미지 인덱스와 그에 대한 호모그래피 행렬을 담은 딕셔너리
        """
        # 모든 이미지 인덱스를 키로 사용하는 딕셔너리 초기화
        H_matrices = {i: None for i in range(len(images))}
        H_matrices[best_index] = np.eye(3, dtype=np.float32)  # 앵커 이미지는 단위 행렬

        # 호모그래피 그래프 생성
        for (pair, kp1, kp2, score, good_matches, H) in match_data:
            # 이미지 인덱스로 변환 (tuple에서 직접 참조)
            img1_idx, img2_idx = pair
            
            # 연결되지 않은 이미지를 연결
            if H_matrices[img1_idx] is not None and H_matrices[img2_idx] is None:
                H_matrices[img2_idx] = H_matrices[img1_idx] @ H
            elif H_matrices[img2_idx] is not None and H_matrices[img1_idx] is None:
                H_matrices[img1_idx] = H_matrices[img2_idx] @ np.linalg.inv(H)

        return H_matrices

    def each_transform(self, images):
        """
        각 이미지에 대해 호모그래피 변환을 적용하여 정렬된 이미지를 반환
        
        Parameters:
        - images (list): 블렌딩할 이미지 리스트
        - H_matrices (dict): 각 이미지의 호모그래피 행렬
        
        Returns:
        - list: 변환된 이미지 리스트
        """
        transformed_images = []
        for idx, H in self.H_matrices.items():
            if H is not None:
                transformed_img = cv2.warpPerspective(images[idx], H, (images[idx].shape[1], images[idx].shape[0]))
                transformed_images.append(transformed_img)
            else:
                transformed_images.append(images[idx])
        return transformed_images


### 실행 

1. Projection

In [31]:
# 카메라 매트릭스 및 회전 행렬 가져오기
Ks, Rs = [], []
for i in range(1, 9):
    K, R = get_camera_matrices(f'./image/problem_3/{i}.jpg')
    Ks.append(K)
    Rs.append(R)

image_list = [img1, img2, img3, img4, img5, img6, img7, img8]

No EXIF data found.
No EXIF data found.
No EXIF data found.
No EXIF data found.
No EXIF data found.
No EXIF data found.
No EXIF data found.
No EXIF data found.


In [32]:
warped_images = []
# 각 이미지에 대해 호모그래피 변환 적용
for i, (img, K, R) in enumerate(zip(image_list, Ks, Rs)):
    # 채널 체크 (회색조 -> 컬러 변환)
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    # 스케일 계산 (focal length / sensor width)
    scale = K[0, 0]
    # Warper 생성
    warper = cv2.PyRotationWarper("cylindrical", scale)
    
    # 워핑 적용
    corner, warped_image = warper.warp(img, K.astype(np.float32), R.astype(np.float32), cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)
    
    # 워핑된 이미지 저장
    warped_images.append(warped_image)
# Plot original and projected images
fig, axes = plt.subplots(3, 2, figsize=(15, 20))
for i, (img, projected_img) in enumerate(zip(image_list, warped_images)):
    # 원본 이미지
    axes[i, 0].imshow(img)
    axes[i, 0].set_title(f"Original Image {i+1}")
    axes[i, 0].axis('off')
    
    # 왜곡된 이미지
    axes[i, 1].imshow(projected_img)
    axes[i, 1].set_title(f"Warped Image {i+1}")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

# 워핑된 이미지로 업데이트
image_list = warped_images


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# 2. SIFT 특징점 검출  시각화 
image_list = warped_images.copy()
kps, dess = [], []
for i, img in enumerate(image_list):
    sift = cv2.SIFT_create()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if img.ndim == 3 else img
    kp, des = sift.detectAndCompute(gray, None)
    kps.append(kp)
    dess.append(des)

# Draw keypoints on the images
keypoint_colcor = []
for i, (img, kp) in enumerate(zip(image_list, kps)):
    img_kp = cv2.drawKeypoints(img, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    keypoint_colcor.append(img_kp)
    
fig, axes = plt.subplots(3, 2, figsize=(20,30))

for i, (img, img_kp) in enumerate(zip(image_list, keypoint_colcor)):
    axes[i, 0].imshow(img)
    axes[i, 0].set_title(f"Original Image {i+1}")
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(img_kp)
    axes[i, 1].set_title(f"Keypoints Image {i+1}")
    axes[i, 1].axis('off')

# plt.show()

In [ ]:
match_data = MatchData()
# 3. 매칭된 키포인트 쌍 및 score 계산
from itertools import combinations
id_list = [i for i in range(len(image_list))]
kps_pairs = combinations(zip(id_list, kps, dess), 2)
for (idx1, kp1, des1), (idx2, kp2, des2) in kps_pairs:
    score, good_matches, H = compute_matching(kp1, des1, kp2, des2)
    match_data.add((idx1, idx2), kp1, kp2, score, good_matches, H)
# 4. 매칭된 키포인트 시각화
def draw_matches(img1, img2, kp1, kp2, good_matches):
    # 매칭된 키포인트 시각화
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    return img_matches
for (pair, kp1, kp2, score, good_matches, H) in match_data.data:
    img1_idx, img2_idx = pair
    img1 = image_list[img1_idx]
    img2 = image_list[img2_idx]
    img_matches = draw_matches(img1, img2, kp1, kp2, good_matches)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(img_matches)
    plt.title(f'Matched Keypoints between Image {img1_idx+1, img2_idx+1} - Score: {score}')
    plt.axis('off')
    plt.show()

In [ ]:
imagesticher = ImageStitcher(blending_method="none")
# 5. 블렌딩 및 투영
stitched_image = imagesticher(match_data, image_list)
# 6. 변형된 이미지 시각화
transformed_images = imagesticher.each_transform(image_list)
for i, img in enumerate(transformed_images):
    plt.figure(figsize=(10, 5))
    plt.imshow(img)
    plt.title(f'Transformed Image {i+1}')
    plt.axis('off')
    plt.show()

In [ ]:
# 5. 최종 이미지 시각화
plt.figure(figsize=(10, 5))
plt.imshow(stitched_image)
plt.title('Stitched Image')
plt.axis('off')
plt.show()